In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import os
import sys
import feather
import numba
import datetime

In [2]:
#this codebook is very slow. it takes approaximately 40 mins.
#change the data_root before run it, if you are sure to have enough time

data_root ='D:\\Programming\\Projects\\notMNIST'
image_size=28

def feather_3d(file,image_size = 28):
    temp = feather.read_dataframe(file)
    width = np.shape(temp.values)[1]
    if width == 1:
        final = temp.values
    else:
        final = np.reshape(temp.values, (-1, image_size, image_size))
    return final

def read_data(dir, filename):
    file = os.path.join(dir, filename + '.feather')
    output = feather_3d(file)
    return output

test_dataset = read_data(data_root, 'test_dataset')
test_labels = read_data(data_root, 'test_labels')
train_dataset = read_data(data_root, 'train_dataset')
train_labels = read_data(data_root, 'train_labels')
valid_dataset = read_data(data_root, 'valid_dataset')
valid_labels = read_data(data_root, 'valid_labels')

In [3]:
#filter blank images

def dataclean(dataset,label,threshold):
    leng=dataset.shape[0]
    indicator=np.ones(leng, dtype=bool)
    temp=0.1
    for i in range(leng):
        temp=dataset[i,:,:].sum()
        if temp >= threshold or temp <= -threshold:
            indicator[i] = 0
    return dataset[indicator,:,:],label[indicator,:]

test_dataset_clean, test_labels_clean = dataclean(test_dataset, test_labels, 378.0)
train_dataset_clean, train_labels_clean = dataclean(train_dataset, train_labels, 378.0)
valid_dataset_clean, valid_labels_clean = dataclean(valid_dataset, valid_labels, 378.0)

print("The obs. in cleaned test data:", test_dataset_clean.shape[0])
print("The obs. in cleaned training data:", train_dataset_clean.shape[0])
print("The obs. in cleaned valid data:", valid_dataset_clean.shape[0])

The obs. in cleaned test data: 11801
The obs. in cleaned training data: 197309
The obs. in cleaned valid data: 19752


In [16]:
@numba.jit('void(f4[:,:,:],i4[:,:],f4)')

def delete_overlap(trainset, trainlabel, smallset, smalllabel):
    leng_s=smallset.shape[0]
    leng_t=trainset.shape[0]
    indicator=np.ones(leng_s, dtype=bool)
    location=[]
    k=0
    for i in range(leng_s):
        location = np.where(trainlabel==smalllabel[i,0])[0]
        for j in location:
            if np.linalg.norm(smallset[i,:,:]-trainset[j,:,:])<=0.27:
                indicator[i] = 0
                k+=1
                if k % 200 == 0:
                    print(k, 'in', i, ' = ', k/i)
            break
    return smallset[indicator,:,:], smalllabel[indicator,:]
test=numba.jit('void(f4[:,:,:],i4[:,:],f4[:,:,:],i4[:,:])')(delete_overlap)

In [ ]:
#filter similar images in valid and test sets.
#applied partial vectorization to accelerate the function though it is still very slow.

def quick_delete(trainset, trainlabel, smallset, smalllabel):
    leng_s=smallset.shape[0]
    indicator=np.ones(leng_s, dtype=bool)
    location=[]
    k=0
    train2d= trainset.reshape(-1, 28*28)
    small2d= smallset.reshape(-1, 28*28)
    for i in range(leng_s):
        location = np.where(trainlabel==smalllabel[i,0])[0]
        if not np.alltrue(np.linalg.norm(train2d[location,:] - small2d[i,:], axis=1)>0.27):
            indicator[i] = 0
            k +=1
            if k % 200 == 0:
                   print(k, 'in', i, ' = ', k/i)
    return smallset[indicator,:,:], smalllabel[indicator,:]

starttime = datetime.datetime.now()

test_dataset_nooverlap, test_labels_nooverlap = quick_delete(train_dataset_clean, train_labels_clean, test_dataset_clean, test_labels_clean)
valid_dataset_nooverlap, valid_labels_nooverlap = quick_delete(train_dataset_clean, train_labels_clean, valid_dataset_clean, valid_labels_clean)

endtime = datetime.datetime.now()
print(endtime-starttime)

D:\Programming\Anaconda3\lib\site-packages\numpy\linalg\linalg.py:2390: RuntimeWarning: invalid value encountered in sqrt
  return sqrt(add.reduce(s, axis=axis, keepdims=keepdims))


In [4]:
#This is the old-version filter.
#Do not run this cell!!
#Do not!

def delete_overlap(trainset, trainlabel, smallset, smalllabel):
    leng_s=smallset.shape[0]
    leng_t=trainset.shape[0]
    indicator=np.ones(leng_s, dtype=bool)
    location=[]
    k=0
    for i in range(leng_s):
        location = np.where(trainlabel==smalllabel[i,0])[0]
        for j in location:
            if np.linalg.norm(smallset[i,:,:]-trainset[j,:,:])<=0.27:
                indicator[i] = 0
                k+=1
                if k % 200 == 0:
                    print(k, 'in', i, ' = ', k/i)
            break
    return smallset[indicator,:,:], smalllabel[indicator,:]

starttime = datetime.datetime.now()

test_dataset_nooverlap, test_labels_nooverlap = delete_overlap(train_dataset_clean, train_labels_clean, test_dataset_clean, test_labels_clean)
valid_dataset_nooverlap, valid_labels_nooverlap = delete_overlap(train_dataset_clean, train_labels_clean, valid_dataset_clean, valid_labels_clean)

print("The obs. in nooverlap test data:", test_dataset_nooverlap.shape[0])
print("The obs. in nooverlap valid data:", valid_dataset_nooverlap.shape[0])

endtime = datetime.datetime.now()
print(endtime-starttime)

The obs. in nooverlap test data: 11801
The obs. in nooverlap valid data: 19752
0:00:38.674584


In [13]:
def save_feather3d(dataset, filename,image_size = 1, force = False):
    dir = os.path.join(data_root, filename + '.feather')
    if os.path.exists(dir) and not force:
        print(filename + '.feather', "already existed")
    else:
        df = pd.DataFrame(dataset.reshape(-1, image_size))
        feather.write_dataframe(df, dir)

save_feather3d(train_dataset_clean, 'train_dataset_final',image_size)
save_feather3d(test_dataset_nooverlap, 'test_dataset_final',image_size)
save_feather3d(valid_dataset_clean, 'valid_dataset_final',image_size)
save_feather3d(train_labels_clean, 'train_labels_final')
save_feather3d(test_labels_nooverlap, 'test_labels_final')
save_feather3d(valid_labels_clean, 'valid_labels_final')

train_dataset_final.feather already existed
valid_dataset_final.feather already existed
train_labels_final.feather already existed
valid_labels_final.feather already existed


In [8]:
type(test_labels[1,0])

numpy.int32